In [13]:
from PIL import Image
import skimage.io
import skimage.filters
import skimage.morphology
from skimage.morphology import disk
from skimage.morphology import square
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os

%matplotlib inline

In [25]:
def get_foci_chloromask(filename, chloro_filename, debug):

    #MJR modified 1/27/25
    #this version relies on the chlorophyll fluorescence image to define the cell mask, should remove any
    #possible artifactual correlation between defining the cell boundary and the presence of foci
    
    #MJR modified 5/12/24
    
    #set debug = True when calling this function to see images of each stage
    #of processing
    
    #parameters (may need to be tuned)
    
    cellmaskthreshfactor = 3.5 #how many standard deviations must a 
    #pixel in the median-filtered image to be confident it's inside a cell
    
    minsigma = 1
    maxsigma = 5
    #essentially smallest and largest features that can be detected
    #using difference-of-gaussians
    
    dog_thresh = 0.002  
    #MJR 1/27/25 changing the threshold
    #dog_thresh_rel = 0.6
    #threshold for minimum intensity needed in peak finding

    
    final_intensity_thresh = 1.2
    #how much does total integrated signal need to rise above background to be counted?
    #in units of std dev of the image
    

    imload = Image.open(filename)
    chloro_imload = Image.open(chloro_filename)
    im = np.array(imload)
    chloro_im = np.array(chloro_imload)
    
    ydim, xdim = im.shape
    if debug:
        print('raw image')
        plt.figure()
        skimage.io.imshow(im)
        plt.show()

    # median filter image (this should remove hot pixels)
    filtersize = 3  #radius of disk used for median filter
    median_im = skimage.filters.median(im, disk(filtersize))
    
    print(type(median_im))

    chloro_median_im = skimage.filters.median(chloro_im, disk(filtersize))

    if debug:
        print(np.max(median_im))
        print(np.min(median_im))
        print(median_im.shape)

    if debug:
        print('median filtered')
        plt.figure()
        skimage.io.imshow(median_im)

    #calculate threshold based on mean and standard deviation ## MJR 1/27/25 not for this anymore: (to get cell mask)
    mu = np.mean(median_im)
    sigma = np.std(median_im)

    chloro_mu = np.mean(chloro_median_im)
    chloro_sigma = np.std(chloro_median_im)

    #MJR 1/27/25 now we use the chlorophyll image
    

    #thresh = mu + cellmaskthreshfactor*sigma
    #changed to constant multiple of mean MJR 5/12/24
    thresh = 1.3*mu
    chloro_thresh = 3.0*chloro_mu  #changed this for the chlorophyll image MJR 1/27/25

    cellmask = chloro_im > chloro_thresh
    


    
    #dilate the cell mask to be sure to get the edges
    cellmask = skimage.morphology.dilation(cellmask, square(3))
    cellmask = skimage.morphology.dilation(cellmask, square(3))
    #cellmask = skimage.morphology.dilation(cellmask, square(3))  #MJR 5/12/24

    threshcomp = lambda t, mask: t*mask
    threshfunc = np.vectorize(threshcomp)
    thresh_im = threshfunc(median_im,cellmask)


    #background subtraction 1/29/25 MJR
    #this should be vectorized...
    tally = 0
    #changed to do median subtraction within cells 
    inmedianlist = []
    outmedianlist = []
    
    for y in range(ydim):
        for x in range(xdim):
            if cellmask[y,x]:
                inmedianlist.append(median_im[y,x])
            else:
                outmedianlist.append(median_im[y,x])
    intense = np.median(inmedianlist)
    outmedian = np.median(outmedianlist)
  
    
    backsub = np.copy(median_im)
    outbacksub = np.copy(thresh_im)
    for y in range(ydim):
        for x in range(xdim):
            if median_im[y,x] > intense:
                backsub[y,x] = backsub[y,x] - intense
            else:
                backsub[y,x] = 0
            if outbacksub[y,x] > outmedian:
                outbacksub[y,x] = outbacksub[y,x] - outmedian
            else:
                outbacksub[y,x] = 0
    
    if debug:
        print('cell mask')
        plt.figure()
        skimage.io.imshow(100*cellmask)
    if debug:
        print('thresholded image')
        plt.figure()
        skimage.io.imshow(thresh_im)

    if debug:
        print('backsub image')
        print(type(intense))
        print(intense)
        plt.figure()
        skimage.io.imshow(backsub)

    
    blobs_dog = skimage.feature.blob_dog(skimage.img_as_float(backsub), min_sigma=minsigma, max_sigma=maxsigma, overlap=0.4, threshold=dog_thresh)
   


    #calculate spot intensity by subtracting local median then summing over a patch
    halfwidth = 4
    results = []
    intensity_thresh = final_intensity_thresh*sigma*(halfwidth+1)**2


    if debug:
        fig,axes = plt.subplots()
        axes.imshow(median_im)
    
    for blob in blobs_dog:
        y, x, r = blob
        #safety check that object is not too close to the edge #MJR 5/12/24
        if x > halfwidth and y > halfwidth and x < xdim - halfwidth - 1 and y < ydim - halfwidth - 1:
            #changed to use median-filtered image here MJR 1/27/25
            subimage = im[int(y)-halfwidth:int(y)+halfwidth,int(x)-halfwidth:int(x)+halfwidth]

            
            #subimage = subimage - np.median(subimage)
            #subimage = median_im[int(x)-halfwidth:int(x)+halfwidth,int(y)-halfwidth:int(y)+halfwidth]
            #MJR 5/12/24 use median filtered image to calculate intensity
            subimage = subimage - np.min(subimage)   #MJR 5/12/24 #removed this 1/27/25 becauae now using median-filtered image
            intensity = np.sum(subimage)
            #print('checking on ', blob)
            #if intensity > intensity_thresh:
            #MJR 1/27/25 modify to check if the center is in the cell mask
            if intensity > intensity_thresh and cellmask[round(y),round(x)]:
                result = y, x, intensity
                if debug:
                    c = plt.Circle((x, y), r, color='lime', linewidth=2, fill=False)
                    axes.add_patch(c)
                    print('validated at: ', x, y, r)
                results.append(result)
            else:
                if debug:
                    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
                    axes.add_patch(c)
                    print('invalidated at: ', x, y, r)
                
    if debug:
        print(results)
        plt.show()

    #for summary results, add up area of cell mask
    cell_area = np.sum(cellmask)
    #calculate total intensity of cells
    #total_intensity = np.sum(backsub)
    #MJR 2/4/25 total_intensity should be calculated from the median-filtered masked imaged w/ median background from non-cell area
    total_intensity = np.sum(outbacksub)
    
    foci_intensity = sum([z for x,y,z in results])
    num_foci = len(results)
    intensity_per_area = foci_intensity/cell_area
    total_intensity_per_area = total_intensity/cell_area
    intensity_adj_total = intensity_per_area/total_intensity_per_area

    return num_foci, foci_intensity, cell_area, total_intensity, intensity_per_area, total_intensity_per_area, intensity_adj_total

In [3]:
def process_directory_chloro(path):
    #modified MJR 1/27/25
    #we assume that a folder exists containing chlorophyll images that has the same name as "path" except "YFP" is
    #replaced by "Chlorophyll". Further we assume that there is a matched image file in that directory that has the same name
    #as the YFP fluorescence image except with "YFP" replaced by "Chlorophyll"
    chloro_path = path.replace('YFP','Chlorophyll')
    files = os.listdir(path)
    #the key function below is a fix to make sure files with single digit IDs
    #get processed first
    files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    analysis = []
    for name in files:
        print(name)
        chloro_name = name.replace('YFP','Chlorophyll')
        filename = path + name
        chloro_filename = chloro_path + chloro_name
        num_foci, foci_intensity, cell_area, total_intensity, intensity_per_area, total_intensity_per_area, intensity_adj_total = get_foci_chloromask(filename, chloro_filename, False)
        analysis.append((name, num_foci, foci_intensity, cell_area, total_intensity, intensity_per_area, total_intensity_per_area, intensity_adj_total))
    return analysis

In [26]:
import csv
analysis = process_directory_chloro('/Users/michaelrust/Downloads/1-1/1-1_Channel_YFP/')
with open('test.csv', 'w', newline ='') as file:
    f = csv.writer(file)
    f.writerows(analysis)

Cyano_Scan_0_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_1_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_2_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_3_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_4_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_5_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_6_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_7_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_8_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_9_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_10_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_11_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_12_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_13_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_14_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_15_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_16_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_17_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_18_Pos_0_YFP.tif
<class 'numpy.ndarray'>
Cyano_Scan_19_Pos_0_YF

In [13]:
pip show scikit-image


Name: scikit-image
Version: 0.24.0
Summary: Image processing in Python
Home-page: https://scikit-image.org
Author: 
Author-email: 
License: Files: *
        Copyright: 2009-2022 the scikit-image team
        License: BSD-3-Clause
        
        Files: doc/source/themes/scikit-image/layout.html
        Copyright: 2007-2010 the Sphinx team
        License: BSD-3-Clause
        
        Files: skimage/feature/_canny.py
               skimage/filters/edges.py
               skimage/filters/_rank_order.py
               skimage/morphology/_skeletonize.py
               skimage/morphology/tests/test_watershed.py
               skimage/morphology/watershed.py
               skimage/segmentation/heap_general.pxi
               skimage/segmentation/heap_watershed.pxi
               skimage/segmentation/_watershed.py
               skimage/segmentation/_watershed_cy.pyx
        Copyright: 2003-2009 Massachusetts Institute of Technology
                   2009-2011 Broad Institute
             

In [13]:
filename='/Users/michaelrust/Downloads/1-1/1-1_Channel_YFP/Cyano_Scan_0_Pos_0_YFP.tif'

In [14]:
imload = Image.open(filename)

In [15]:
type(imload)

PIL.TiffImagePlugin.TiffImageFile

In [16]:
im = np.array(imload)

In [17]:
type(im)

numpy.ndarray

In [19]:
im[10,10]

np.uint16(714)

In [20]:
filtersize = 3  #radius of disk used for median filter
median_im = skimage.filters.median(im, disk(filtersize))

In [21]:
type(median_im)

numpy.ndarray

In [22]:
median_im[10,10]

np.uint16(701)

In [23]:
test = skimage.img_as_float(median_im)

In [24]:
type(test)

numpy.ndarray

In [25]:
test[10,10]

np.float64(0.010696574349584192)

In [28]:
np.max(test)

np.float64(0.02238498512245365)